In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

from issue_tagging_bot.issue_data import Stage1PreprocData, Stage2PreprocData

In [3]:
stage2 = Stage2PreprocData()
train_set, val_set, test_set = stage2.to_datasets()

In [4]:
def only_nixos_labels(dataset: tf.data.Dataset) -> tf.data.Dataset:                                                                                                                     
    return dataset.map(lambda issue_body, labels, issue_num: (issue_body, labels[-1]))                                                                                                                           

In [18]:
train_set = train_set.shuffle(buffer_size=100000, seed=42, reshuffle_each_iteration=True)

train_set_without_issue_num = only_nixos_labels(train_set).batch(10)
val_set_without_issue_num = only_nixos_labels(val_set).batch(1)
test_set_without_issue_num = only_nixos_labels(test_set).batch(1)

In [19]:
train_set_without_issue_num

<BatchDataset shapes: ((None, 1000), (None,)), types: (tf.int8, tf.int64)>

In [20]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

#model.summary()

In [21]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
history = model.fit(train_set_without_issue_num, epochs=10, validation_data=val_set_without_issue_num)

Train for 1257 steps, validate for 1600 steps
Epoch 1/10
1257/1257 [==============================] - 9s 7ms/step - loss: 9.0888 - accuracy: 0.8602 - val_loss: 6.4101 - val_accuracy: 0.9237
Epoch 2/10
1257/1257 [==============================] - 31s 25ms/step - loss: 1.6084 - accuracy: 0.8695 - val_loss: 0.7934 - val_accuracy: 0.9219
Epoch 3/10
1257/1257 [==============================] - 10s 8ms/step - loss: 0.4612 - accuracy: 0.8979 - val_loss: 0.2910 - val_accuracy: 0.9237
Epoch 4/10
1257/1257 [==============================] - 9s 7ms/step - loss: 0.2768 - accuracy: 0.9247 - val_loss: 0.2936 - val_accuracy: 0.9237
Epoch 5/10
1257/1257 [==============================] - 15s 12ms/step - loss: 0.2856 - accuracy: 0.9246 - val_loss: 0.3039 - val_accuracy: 0.9237
Epoch 6/10
1257/1257 [==============================] - 12s 10ms/step - loss: 0.2783 - accuracy: 0.9247 - val_loss: 0.3470 - val_accuracy: 0.9237
Epoch 7/10
1257/1257 [==============================] - 9s 7ms/step - loss: 0.2815 

In [23]:
model.evaluate(test_set_without_issue_num)

1500/1500 [==============================] - 3s 2ms/step - loss: 0.2562 - accuracy: 0.9320


[0.25615801994372783, 0.932]

In [79]:
predictions = model.predict(test_set_without_issue_num).reshape(-1)
predictions = np.array(list(map(lambda val: 1 if val > 0.1 else 0, predictions)))

In [80]:
real_vals = np.array(list(test_set_without_issue_num.map(lambda x, p: p).as_numpy_iterator())).reshape(-1)

In [81]:
confusion_matrix(real_vals, predictions)

array([[850, 548],
       [ 58,  44]])

In [39]:


# TODO: confusion matrix function can't handle a mix of int and float targets

array([0.07405987, 0.08968125, 0.16573511, ..., 0.05871441, 0.11020781,
       0.11297136], dtype=float32)

In [37]:
#confusion_matrix(blaha, predictions)

ValueError: Classification metrics can't handle a mix of binary and continuous targets